## How to do it...

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

housing_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
path = tf.keras.utils.get_file(housing_url.split('/')[-1], housing_url)

columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = pd.read_table(path, delim_whitespace=True, header=None, names=columns)

np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = train['MEDV']
train.drop('MEDV', axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = test['MEDV']
test.drop('MEDV', axis=1, inplace=True)

2022-01-21 08:49:29.676537: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [7]:
learning_rate = 0.05
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
    
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    
    return input_function

def define_feature_columns(data_df, categorical_cols, numeric_cols):
    feature_columns = []
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
        
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
        
    return feature_columns

categorical_cols = ['CHAS', 'RAD']
numeric_cols = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_columns = define_feature_columns(data, categorical_cols, numeric_cols)

train_input_fn = make_input_fn(train, y_train, num_epochs=1400)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

In [8]:
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpkzyev3nr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Calling model_fn.


/home/wil/tensorflow/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


2022-01-21 09:01:03.477876: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-21 09:01:03.478395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:01:03.478764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-21 09:01:03.478809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), 

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


...

INFO:tensorflow:loss = 612.27734, step = 0


2022-01-21 09:01:05.289082: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


INFO:tensorflow:global_step/sec: 275.018
INFO:tensorflow:loss = 57.77643, step = 100 (0.364 sec)
INFO:tensorflow:global_step/sec: 355.421
INFO:tensorflow:loss = 43.788067, step = 200 (0.281 sec)
INFO:tensorflow:global_step/sec: 352.424
INFO:tensorflow:loss = 46.79715, step = 300 (0.284 sec)
INFO:tensorflow:global_step/sec: 361.076
INFO:tensorflow:loss = 50.137627, step = 400 (0.277 sec)
INFO:tensorflow:global_step/sec: 361.315
INFO:tensorflow:loss = 33.98306, step = 500 (0.277 sec)
INFO:tensorflow:global_step/sec: 359.839
INFO:tensorflow:loss = 40.513027, step = 600 (0.278 sec)
INFO:tensorflow:global_step/sec: 362.275
INFO:tensorflow:loss = 41.30123, step = 700 (0.276 sec)
INFO:tensorflow:global_step/sec: 354.927
INFO:tensorflow:loss = 40.473793, step = 800 (0.282 sec)
INFO:tensorflow:global_step/sec: 361.245
INFO:tensorflow:loss = 40.663395, step = 900 (0.277 sec)
INFO:tensorflow:global_step/sec: 362.783
INFO:tensorflow:loss = 34.62506, step = 1000 (0.276 sec)
INFO:tensorflow:global_s

2022-01-21 09:01:13.639438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:01:13.639670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-21 09:01:13.639717: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:01:13.639921: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:01:13.640096: I tensorflow/core/common

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2800: /tmp/tmpkzyev3nr/model.ckpt-2800
{'average_loss': 32.782837, 'label/mean': 22.048513, 'loss': 32.782837, 'prediction/mean': 21.307537, 'global_step': 2800}


In [10]:
def create_interactions(interactions_list, buckets=5):
    interactions = list()
    for (a, b) in interactions_list:
        interactions.append(tf.feature_column.crossed_column([a, b], hash_bucket_size=buckets))
    return interactions

derived_feature_columns = create_interactions([['RM', 'LSTAT']])
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns+derived_feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmphgdr5us2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:ten

2022-01-21 09:06:26.573683: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:06:26.573932: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-21 09:06:26.573976: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:06:26.574202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:06:26.574408: I tensorflow/core/common

INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmphgdr5us2/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 630.23883, step = 0
INFO:tensorflow:global_step/sec: 233.531
INFO:tensorflow:loss = 69.27646, step = 100 (0.429 sec)
INFO:tensorflow:global_step/sec: 304.013
INFO:tensorflow:loss = 51.57306, step = 200 (0.329 sec)
INFO:tensorflow:global_step/sec: 298.699
INFO:tensorflow:loss = 47.784103, step = 300 (0.335 sec)
INFO:tensorflow:global_step/sec: 299.853
INFO:tensorflow:loss = 46.912075, step = 400 (0.334 sec)
INFO:tensorflow:global_step/sec: 302.546
INFO:tensorflow:loss = 40.785793, step = 500 (0.330 sec)
INFO:tensorflow:global_step/sec: 299.408
INFO:tensorflow:loss = 38.48613, step = 600 (0.334 sec)
INFO:tensorflow:global_step/sec: 297.563
INFO:tensorflow:loss = 39.708355, step = 700 (0.336 sec)
INFO:tensorflow:global_step/sec: 296.948
INFO:tens

2022-01-21 09:06:37.103873: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:06:37.104106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-21 09:06:37.104156: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:06:37.104519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:06:37.104702: I tensorflow/core/common

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2800: /tmp/tmphgdr5us2/model.ckpt-2800
{'average_loss': 31.328875, 'label/mean': 22.048513, 'loss': 31.328875, 'prediction/mean': 21.338617, 'global_step': 2800}


In [12]:
def dicts_to_preds(pred_dicts):
    return np.array([pred['predictions'] for pred in pred_dicts])

preds = dicts_to_preds(linear_est.predict(test_input_fn))
print(preds)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmphgdr5us2/model.ckpt-2800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[24.65097  ]
 [30.475254 ]
 [19.849728 ]
 [21.614136 ]
 [17.876171 ]
 [16.826645 ]
 [18.219368 ]
 [12.463432 ]
 [23.554424 ]
 [25.291218 ]
 [20.821156 ]
 [17.08128  ]
 [28.582363 ]
 [16.866581 ]
 [19.730692 ]
 [21.017778 ]
 [24.091114 ]
 [22.82893  ]
 [24.578596 ]
 [22.67957  ]
 [26.487698 ]
 [25.193134 ]
 [22.903671 ]
 [19.270441 ]
 [20.979124 ]
 [11.677001 ]
 [17.457994 ]
 [21.350342 ]
 [20.247923 ]
 [ 5.791603 ]
 [13.066736 ]
 [10.124878 ]
 [19.751698 ]
 [19.134785 ]
 [15.171392 ]
 [33.53774  ]
 [27.49646  ]
 [28.843071 ]
 [33.98083  ]
 [33.80422  ]
 [34.73173  ]
 [15.667826 ]
 [20.151356 ]
 [23.915293 ]
 [36.318638 ]
 [26.219662 ]
 [32.44004  ]
 [24.488693 ]
 [20.141724 ]
 [22.567919 ]
 [27.22725  ]
 [24.619722 ]
 [23

2022-01-21 09:07:43.433798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:07:43.434048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-21 09:07:43.434095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:07:43.434321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 09:07:43.434518: I tensorflow/core/common